In [209]:
import pandas as pd, openpyxl
import mysql.connector.connection
import os, sys, json
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
# Configuracion ruta json
with open("rutas.json", "r") as archivo:
    rutas = json.load(archivo)

Configuracion Principal

In [210]:
month = 4
year = 2025

In [211]:
date = f"{month}-{year}"
date = datetime.strptime(date, "%m-%Y")
month_str = datetime.strftime(date, "%B")
file_name = f"{month_str} {year}.xlsx"
file_name

'April 2025.xlsx'

In [212]:
df = pd.read_excel(f"{rutas['input']}{file_name}")
df.head()

,Legajo,Documento,Apellido,Nombre,Basico,D. Enfermedad,Vacaciones,Feriado,Licencia,D. Mudanza,...,Falta Inj.,Hs Extras,Dep. Judicial,Prepaga,Enlazados,Ant. de Sueldos,Adic. Ventas,OBSERVACIONES,Fecha baja,Fecha alta
0,1365,44059638,ABARCA,NOELIA AGUSTINA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,1964,38417005,ABBONA,MATIAS ANDRES,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,1906,42976152,ACUÑA,IGNACIO ESTEBAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,2078,44746629,AGUILERA,ANDREA MACARENA,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,1907,44436947,AGUILERA,EDGAR GABRIEL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


PROCESAMIENTO DE DATOS

El fin de este procesamiento es generar un dataframe en el cual me haga aparezca por asesor la cantidad de licencias o excepciones que tiene, compararlo con el codificado.



In [213]:
# limpieza de dataframe
df = df[["Legajo", "Hs Extras", "Dep. Judicial", "Prepaga", "Enlazados", "Ant. de Sueldos", "Adic. Ventas"]]
df.head()

,Legajo,Hs Extras,Dep. Judicial,Prepaga,Enlazados,Ant. de Sueldos,Adic. Ventas
0,1365,NaN,NaN,NaN,NaN,NaN,NaN
1,1964,NaN,NaN,NaN,NaN,NaN,NaN
2,1906,NaN,NaN,NaN,NaN,NaN,NaN
3,2078,NaN,NaN,NaN,NaN,NaN,NaN
4,1907,NaN,NaN,NaN,NaN,NaN,NaN


Extraccion de informacion de la tabla payroll

Extraemos los datos de payoll para la identificacion de licencias y otros dias de ausencia. De este reporte solamente se sacara la informacion desde la columna d. enfermedad hasta la columna Hs Extras

In [214]:
def conexion_db(host, user, password, database):
    try:
        cnx = mysql.connector.connect(host= host, user=user, password=password, database=database)
        print("Conexion exitosa a base de datos")
        return cnx
    except Exception as conection_error:
        print(f"Se produjo un error al intentar conectarse a base de datos: \n{conection_error}")
def extraccion_de_datos (cnx, query):
    try:
        cursor = cnx.cursor()
        cursor.execute(query)
        resultado = cursor.fetchall()
        if len(resultado) == 0:
            print("Advertencia. La consulta no trajo informacion") 
        cursor.close()
        cnx.close()
        print("Se extrajo la informacion correctamente de base de datos")
        return resultado
    except Exception as extract_error:
        print(f"Se produjo un error al extraer la informacion de base de datos {extract_error}")

In [215]:
# Consulta de extraccion payroll
query= f"""
SELECT 
legajo, 
fecha, 
codigo, 
descripcion,
horas_programadas, 
horas_trabajadas
FROM payroll  WHERE MONTH(fecha) = {month} AND YEAR(fecha) = {year}"""
columnas = ["legajo", "fecha", "codigo", "descripcion", "horas_programadas", "horas_trabajadas"]

In [216]:
host = os.getenv('DB_HOST')
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
cnx = conexion_db(host, user, password, "db_omnia")
resultado = extraccion_de_datos(cnx, query)
df_payroll = pd.DataFrame(resultado, columns=columnas)
df_payroll.head()
 

Conexion exitosa a base de datos
Se extrajo la informacion correctamente de base de datos


,legajo,fecha,codigo,descripcion,horas_programadas,horas_trabajadas
0,1098,2025-04-01,P,Presente,8.0,7.93
1,1085,2025-04-01,P,Presente,7.5,8.05
2,1087,2025-04-01,P,Presente,7.5,7.57
3,1213,2025-04-01,P,Presente,7.5,7.55
4,987,2025-04-01,P,Presente,7.5,7.53


In [217]:
codificado = {
    "Feriado" : 8,
    "Licencia Maternidad": 409,  # LICENCIA POR MATERNIDAD
    "Licencia Fallecimiento Familiar": 56,  # FALLECIMIENTO
    "Licencia Matrimonio": 55,  # MATRIMONIO
    "Certificado Estudio": 62,  # ESTUDIO
    "Licencia Médica": 65,  # ENFERMEDAD
    "Día de Trámite": 51,  # DIA DE TRAMITE
    "Día No Laborable": 8,  # FERIADO
    "Mudanza": 54,
    "Suspensión": 50,
    "Ausente sin aviso": 303, # Consultar
    "Ausencia sin aviso": 303, # Consultar
    "Presente Horario Autorizado": 303,  # PRESENTISMO
    "Prepaga" : 654,
    "Enlazados" : 733,
    "Ant. de Sueldo" : 610,
    "Adic. Ventas" : 45

}



In [218]:
# Hay que realizar un conteo de codigos cuando estos sean los que queremos
""" Se puede realizar tambien una comparacion mediante una cadena de valores """
# iteramos por filas
info_recolectada_payroll = []
for index, row in df_payroll.iterrows():
    if row['descripcion'] in codificado:
        descripcion = row['descripcion'] 
        info_recolectada_payroll.append({
            "Legajo" : row['legajo'],
            "Descripcion" : descripcion,
            "Tipo" : codificado[descripcion],
            "Cantidad" : 1
        })
df_borrar = pd.DataFrame(info_recolectada_payroll)
df_borrar = df_borrar.groupby(by=["Legajo", "Tipo"])["Cantidad"].sum().reset_index()
df_borrar.to_excel(f"{rutas['Output']}procesado database.xlsx", index=False)
df_borrar.head()


,Legajo,Tipo,Cantidad
0,462,8,1
1,462,303,1
2,505,303,6
3,508,303,2
4,512,303,6


In [219]:
resultado = []
for i, fila in df.iterrows():
    for col in df.columns[1:]:  # Salteamos la primera columna (ID)
        if pd.notna(fila[col]):
            resultado.append({
                "Legajo" : fila['Legajo'],
                "Tipo" : col,
                "Cantidad" : fila[col]
            })

df_resultado = pd.DataFrame(resultado)
if len(df_resultado) > 0:
    df_resultado['Legajo'] = df_resultado['Legajo'].astype(int)
    df_resultado['Cantidad'] = df_resultado['Cantidad'].astype(int)
    
df_resultado.head()


,Legajo,Tipo,Cantidad
0,1980,Prepaga,2
1,1980,Enlazados,35
2,1673,Prepaga,8
3,1770,Prepaga,9
4,1188,Prepaga,2


In [220]:
codificado_excel = []
for index, row in df_resultado.iterrows():
    if row['Tipo'] in codificado:
        tipo = row['Tipo']
        codificado_excel.append({
            "Legajo" : row['Legajo'],
            "Tipo" : codificado[tipo],
            "Cantidad" : row['Cantidad']
        })

df_resultado = pd.DataFrame(codificado_excel)
df_resultado.head()
df_resultado.to_excel(f"{rutas['Output']}procesado excel.xlsx", index=False)

In [221]:
df_final = pd.concat([df_borrar, df_resultado],axis=0)
df_final.to_excel(f"{rutas['Output']}concatenado.xlsx", index= False)